In [8]:
# load required packages
import pymysql
import decouple
import requests
import json
import os
import re


In [9]:
# check db connection
config = decouple.AutoConfig(' ')

user = config('AWS_MY_DB_ADMIN_USER')
host = config('AWS_MY_DB_ADMIN_HOST')
dbname = config('AWS_MY_DB_ADMIN_DBNAME')
pw = config('AWS_MY_DB_ADMIN_PW')
port = int(config('AWS_MY_DB_ADMIN_PORT'))

con = pymysql.connect(host=host,
                      user=user, 
                      database=dbname,
                      password=pw,
                      port=port
                     )
cur = con.cursor()

cur.execute('SHOW TABLES')
for table_name in cur:
   print(table_name)

con.close()

if (con.open != True):
    print("Connection is closed")

('EQR',)
('EWJ',)
('GLD',)
('IWM',)
('MCHI',)
('QQQ',)
('SLV',)
('SPY',)
('UNG',)
('USO',)
('VGK',)
('VIXM',)
('VWO',)
('imdb_boxOffice',)
('imdb_details',)
('imdb_details_extd',)
('imdb_top250_movies',)
('imdb_top250_shows',)
Connection is closed


In [11]:
# scrape data
path = config('PATH_MY_MAIN_DATA')
symbols = ['SPY', 'USO', 'UNG', 'GLD',
           'SLV', 'QQQ', 'IWM', 'MCHI',
           'EWJ', 'VWO', 'VGK', 'EQR',
           'VIXM']
symbols = ['AAL']

for symbol in symbols:
    api_key = config('ALPHAVANTAGE_API_KEY')
    size = 'full' # compact returns last 100, full returns 20+ years
    url_base = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='
    url = url_base + symbol + '&outputsize=' + size + '&apikey=' + api_key

    f = path + "/" + symbol + ".json"

    if os.path.isfile(f) == False:
        r = requests.get(url)
        data = r.json()
        data = json.dumps(data)
        with open(f, "w") as outfile:
            outfile.write(data)
    
    

#print(data)

In [320]:
# create db tables
def checkTableExists(dbcon, tablename):
    dbcur = dbcon.cursor()
    dbcur.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(tablename.replace('\'', '\'\'')))
    if dbcur.fetchone()[0] == 1:
        dbcur.close()
        return True

    dbcur.close()
    return False

con = pymysql.connect(host=host,
                      user=user, 
                      database=dbname,
                      password=pw,
                      port=port
                     )

cur = con.cursor()

xx_list = []
for symbol in symbols:
    x = checkTableExists(con, symbol)
    if x == False:
        qstr = ("CREATE TABLE " +
                str(symbol) +
                "  (date INT(10) NOT NULL, open DOUBLE, high DOUBLE, " +
                "low DOUBLE, close DOUBLE, adjclose DOUBLE, " +
                "volume INT(10), dividend DOUBLE, splits DOUBLE, " +
                "PRIMARY KEY (date))")
        
        cur.execute(qstr)
    
        cur.execute('SHOW COLUMNS FROM {}'.format(symbol))
        xx = cur.fetchall()
        xx_list.append(xx)

con.close()

if (con.open != True):
    print("Connection is closed")


(('date', 'int', 'NO', 'PRI', None, ''), ('open', 'double', 'YES', '', None, ''), ('high', 'double', 'YES', '', None, ''), ('low', 'double', 'YES', '', None, ''), ('close', 'double', 'YES', '', None, ''), ('adjclose', 'double', 'YES', '', None, ''), ('volume', 'int', 'YES', '', None, ''), ('dividend', 'double', 'YES', '', None, ''), ('splits', 'double', 'YES', '', None, ''))
(('date', 'int', 'NO', 'PRI', None, ''), ('open', 'double', 'YES', '', None, ''), ('high', 'double', 'YES', '', None, ''), ('low', 'double', 'YES', '', None, ''), ('close', 'double', 'YES', '', None, ''), ('adjclose', 'double', 'YES', '', None, ''), ('volume', 'int', 'YES', '', None, ''), ('dividend', 'double', 'YES', '', None, ''), ('splits', 'double', 'YES', '', None, ''))
(('date', 'int', 'NO', 'PRI', None, ''), ('open', 'double', 'YES', '', None, ''), ('high', 'double', 'YES', '', None, ''), ('low', 'double', 'YES', '', None, ''), ('close', 'double', 'YES', '', None, ''), ('adjclose', 'double', 'YES', '', None,

In [326]:
# connect to db
con = pymysql.connect(host=host,
                      user=user, 
                      database=dbname,
                      password=pw,
                      port=port
                     )

cur = con.cursor()

# load data into db
path = config('PATH_MY_MAIN_DATA')
symbols = ['SPY', 'USO', 'UNG', 'GLD',
           'SLV', 'QQQ', 'IWM', 'MCHI',
           'EWJ', 'VWO', 'VGK', 'EQR',
           'VIXM']

for symbol in symbols:
    file = path + "/" + symbol + ".json"
    f = open(file)
    data = json.load(f)
    keys = list(data["Time Series (Daily)"].keys())

    for key in keys:
        value_list = list(data["Time Series (Daily)"][key].values())
        date = int(re.sub(r'[^0-9]', '', key))
        o = float(value_list[0])
        h = float(value_list[1])
        l = float(value_list[2])
        c = float(value_list[3])
        ac = float(value_list[4])
        v = int(value_list[5])
        d = float(value_list[6])
        s = float(value_list[7])
        
        
        qstr = ("SELECT date FROM " + symbol + " WHERE date= %s")
        cur.execute(qstr, date)
        xx = cur.fetchall()
        
        if (len(xx) == 0):
            try:
                qstr = ("INSERT INTO " + str(symbol) +
                        " (date, open, high, low, close, adjclose, volume, dividend, splits) " +
                        " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)")
                cur.execute(qstr, (date, o, h, l, c, ac, v, d, s))
                con.commit()
                print("Successfully inserted {} into {} table".format(date, symbol))
                
            except pymysql.Error as exc:
                print("Error inserting {} into {} table \n {}").format(date, symbol, exc)
                
        else:
            print("{} already present in {} table".format(date, symbol))
        
        
con.close()



Successfully inserted 20221110 into SPY table
20221109 already present in SPY table
20221108 already present in SPY table
20221107 already present in SPY table
20221104 already present in SPY table
20221103 already present in SPY table
20221102 already present in SPY table
20221101 already present in SPY table
20221031 already present in SPY table
20221028 already present in SPY table
20221027 already present in SPY table
20221026 already present in SPY table
20221025 already present in SPY table
20221024 already present in SPY table
20221021 already present in SPY table
20221020 already present in SPY table
20221019 already present in SPY table
20221018 already present in SPY table
20221017 already present in SPY table
20221014 already present in SPY table
20221013 already present in SPY table
20221012 already present in SPY table
20221011 already present in SPY table
20221010 already present in SPY table
20221007 already present in SPY table
20221006 already present in SPY table
2022

KeyboardInterrupt: 